# HW1.1: Dictionary-based Tokenization


In this exercise, you are to implement a dictionary-based word segmentation algorithm. There are two Python functions that you need to complete:
<br>
* maximal_matching
* backtrack
</br>

Also, you have to find how to use word_tokenize() in PythaiNLP along with customer_dict by yourselves.

In [1]:
%pip install -qq pythainlp
%pip install -qq marisa_trie
from pythainlp.tokenize import word_tokenize
from pythainlp.corpus import get_corpus
from marisa_trie import Trie

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## Part 1) Maximal Matching from PythaiNLP

### Create a toy dictionary to test the algorithm

This is based on the example shown in the lecture.
You will tokenize the following text string: "ไปหามเหสี!"
The toy dictionary provided in this exercise includes all the charaters, syllables, and words that appear that the text string.

In [2]:
thai_vocab = ["ไ", "ป", "ห", "า", "ม", "เ", "ห", "ส", "ี", "ไป", "หา", "หาม", "เห", "สี", "มเหสี", "!"]
input_text = "ไปหามเหสี!"

### Example Dictionary

Write the `word_tokenize` function of PyThaiNLP with a custom dictionary above and using:
1. Longest matching algorithm `longest`
2. Maximal-matching algorithm `newmm`

Study `word_tokenize()` from PythaiNLP in the link below. Note: `custom_dict` will accept Trie structures as `Trie(iterable)`.

https://pythainlp.org/docs/5.0/api/tokenize.html#pythainlp.tokenize.word_tokenize

In [3]:
####FILL CODE HERE####
custom_dict = Trie(thai_vocab)
print("Longest matching")
print(f'\tTokenize: {word_tokenize(input_text, custom_dict=custom_dict, engine="longest")}')

print("Maximal matching")
print(f'\tTokenize: {word_tokenize(input_text, custom_dict=custom_dict, engine="newmm")}')

######################

Longest matching
	Tokenize: ['ไป', 'หาม', 'เห', 'สี', '!']
Maximal matching
	Tokenize: ['ไป', 'หา', 'มเหสี', '!']


## Part 2) Maximal Matching from Scratch

### Maximal matching
Complete the maximal matching function below with dynamic programming to tokenize the input text and output the 2D numerical array shown in class.

In [4]:
from math import inf  # infinity


def maximal_matching(c: str | list[chr]) -> list[list[int]]:
    # Create custom_dict
    custom_dict = Trie(thai_vocab)

    # Initialize an empty 2D list
    d = [[None] * len(c) for _ in range(len(c))]

    ####FILL CODE HERE####
    # Initialize the first row
    for column in range(len(c)):
        d[0][column] = 1 if c[0 : column + 1] in custom_dict else inf

    # Fill in the rest of the table
    for row in range(1, len(c)):
        # Reset the row
        d[row][:row] = [None] * row
        for column in range(row, len(c)):
            # Check if the current word is in the dictionary
            current_word = c[row : column + 1]
            if current_word in custom_dict:
                # If the current word is in the dictionary, the cost is 1 + the minimum cost of the previous row
                d[row][column] = 1 + min([d[k][row - 1] for k in range(0, row)])
            else:
                # If the current word is not in the dictionary, the cost is infinity
                d[row][column] = inf
    ######################

    return d

### Backtracking
Complete the backtracking function below to find the tokenzied words.
It should return a list containing a pair of the beginning position and the ending position of each word.
In this example, it should return:
<br>
[(0, 1),(2, 3),(4, 8),(9, 9)]
<br>
#### Each pair contains the position of each word as follows:
(0, 1) ไป
<br>
(2, 3) หา
<br>
(4, 8) มเหสี
<br>
(9, 9) !


In [5]:
def backtrack(d: list[list[int]]) -> list[tuple[int, int]]:
    eow = len(d) - 1  # End of Word position
    word_pos = []  # Word position
    ####FILL CODE HERE####

    while eow >= 0:
        for start in range(eow + 1):
            if d[start][eow] != inf and d[start][eow] is not None:
                # Found a valid token boundary
                word_pos.append((start, eow))
                eow = start - 1  # Move to the previous segment
                break

    ######################
    word_pos.reverse()
    return word_pos

### Test your maximal matching algorithm on a toy dictionary

Expected output:
```
[1   ,    1,  inf,  inf,  inf,  inf,  inf,  inf,  inf, inf] ไ
[None,    2,  inf,  inf,  inf,  inf,  inf,  inf,  inf, inf] ป
[None, None,    2,    2,    2,  inf,  inf,  inf,  inf, inf] ห
[None, None, None,    3,  inf,  inf,  inf,  inf,  inf, inf] า
[None, None, None, None,    3,  inf,  inf,  inf,    3, inf] ม
[None, None, None, None, None,    3,    3,  inf,  inf, inf] เ
[None, None, None, None, None, None,    4,  inf,  inf, inf] ห
[None, None, None, None, None, None, None,    4,    4, inf] ส
[None, None, None, None, None, None, None, None,    5, inf]  ี
[None, None, None, None, None, None, None, None, None,   4] !
```

In [6]:
input_text = "ไปหามเหสี!"
out = maximal_matching(input_text)
for i in range(len(out)):
    print(out[i], input_text[i])

[1, 1, inf, inf, inf, inf, inf, inf, inf, inf] ไ
[None, 2, inf, inf, inf, inf, inf, inf, inf, inf] ป
[None, None, 2, 2, 2, inf, inf, inf, inf, inf] ห
[None, None, None, 3, inf, inf, inf, inf, inf, inf] า
[None, None, None, None, 3, inf, inf, inf, 3, inf] ม
[None, None, None, None, None, 3, 3, inf, inf, inf] เ
[None, None, None, None, None, None, 4, inf, inf, inf] ห
[None, None, None, None, None, None, None, 4, 4, inf] ส
[None, None, None, None, None, None, None, None, 5, inf] ี
[None, None, None, None, None, None, None, None, None, 4] !


### Test your backtracking algorithm on a toy dictionary
Compare your results with the result from PyThaiNLP `newmm`.

Expected output:
<br>
ไป|หา|มเหสี|!

In [7]:
def print_tokenized_text(d, input_text):
    tokenized_text = []
    for pos in backtrack(d):
        # print(pos)
        tokenized_text.append(input_text[pos[0] : pos[1] + 1])

    print("|".join(tokenized_text))


print_tokenized_text(out, input_text)

ไป|หา|มเหสี|!


### <font color=blue>Question 1</font>
Using your maximal matching code with the toy dictionary, how many “words” did you get when tokenizing this input text.

Answer this question in question #1 in MyCourseVille. Also print out the answer in this notebook as well.

In [8]:
input_text = "ไปหาหมามเหสีมาหาม!"

out = maximal_matching(input_text)
print(f"Input text: {input_text}")
print(f"Output positions: {backtrack(out)}")
print(f"Token amount: {len(backtrack(out))}")
print_tokenized_text(out, input_text)

Input text: ไปหาหมามเหสีมาหาม!
Output positions: [(0, 1), (2, 3), (4, 4), (5, 5), (6, 6), (7, 11), (12, 12), (13, 13), (14, 16), (17, 17)]
Token amount: 10
ไป|หา|ห|ม|า|มเหสี|ม|า|หาม|!


## Part 3) Your Maximal Matching with Real Dictionary

For UNIX-based OS users, the following cell will download a dictionary (it's just a list of thai words). Alternatively, you can download it from this link: https://raw.githubusercontent.com/PyThaiNLP/pythainlp/dev/pythainlp/corpus/words_th.txt

In [9]:
!wget https://raw.githubusercontent.com/PyThaiNLP/pythainlp/dev/pythainlp/corpus/words_th.txt

--2025-01-08 13:47:43--  https://raw.githubusercontent.com/PyThaiNLP/pythainlp/dev/pythainlp/corpus/words_th.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8001::154, 2606:50c0:8003::154, 2606:50c0:8000::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8001::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1519589 (1.4M) [text/plain]
Saving to: ‘words_th.txt.4’

words_th.txt.4      100%[===================>]   1.45M  --.-KB/s    in 0.06s   

2025-01-08 13:47:44 (24.4 MB/s) - ‘words_th.txt.4’ saved [1519589/1519589]



In [10]:
with open("words_th.txt", encoding="utf-8-sig") as f:
    thai_vocab = f.read().splitlines()
print("Vocab size:", len(thai_vocab))
print(thai_vocab[:10])

thai_vocab.extend(["ๆ", "!"])

Vocab size: 62077
['ก ข ไม่กระดิกหู', 'ก.', 'ก.ค.', 'ก.ต.', 'ก.ป.ส.', 'ก.พ.', 'ก.พ.ด.', 'ก.ม.', 'ก.ย.', 'ก.ย']


### Part 3.1) The output of **YOUR** maximal matching algoithm on the new dictionary
Expected output:
```
[ inf,    1,  inf,    1,  inf,  inf,  inf,  inf,  inf] ไ
[None,  inf,  inf,  inf,  inf,  inf,  inf,  inf,  inf] ป
[None, None,  inf,    2,    2,  inf,  inf,  inf,  inf] ห
[None, None, None,  inf,  inf,  inf,  inf,  inf,  inf] า
[None, None, None, None,  inf,  inf,  inf,  inf,    2] ม
[None, None, None, None, None,  inf,    3,  inf,  inf] เ
[None, None, None, None, None, None,  inf,  inf,  inf] ห
[None, None, None, None, None, None, None,  inf,    4] ส
[None, None, None, None, None, None, None, None,  inf] ี
```
### Expected tokenized text
ไปหา|มเหสี

_Question: Why are the resulting tokens different?_

In [11]:
input_text = "ไปหามเหสี"
out = maximal_matching(input_text)
for i in range(len(out)):
    print(out[i], input_text[i])

print_tokenized_text(out, input_text)

[inf, 1, inf, 1, inf, inf, inf, inf, inf] ไ
[None, inf, inf, inf, inf, inf, inf, inf, inf] ป
[None, None, inf, 2, 2, inf, inf, inf, inf] ห
[None, None, None, inf, inf, inf, inf, inf, inf] า
[None, None, None, None, inf, inf, inf, inf, 2] ม
[None, None, None, None, None, inf, 3, inf, inf] เ
[None, None, None, None, None, None, inf, inf, inf] ห
[None, None, None, None, None, None, None, inf, 4] ส
[None, None, None, None, None, None, None, None, inf] ี
ไปหา|มเหสี


### <font color=blue>Question 2</font>
Using your maximal matching algorithm and the actual Thai dictionary, how many “words” did you get when tokenizing this input text.

Answer this question in question #2 in MyCourseVille. Also print out the answer in this notebook as well.

In [12]:
input_text = "ประเทศไทยรวมเลือดเนื้อชาติเชื้อไทยเป็นประชารัฐไผทของไทยทุกส่วนอยู่ดำรงคงไว้ได้ทั้งมวลด้วยไทยล้วนหมายรักสามัคคี"

out = maximal_matching(input_text)
print(f"Input text: {input_text}")
print(f"Output positions: {backtrack(out)}")
print(f"Token amount: {len(backtrack(out))}")
print_tokenized_text(out, input_text)

Input text: ประเทศไทยรวมเลือดเนื้อชาติเชื้อไทยเป็นประชารัฐไผทของไทยทุกส่วนอยู่ดำรงคงไว้ได้ทั้งมวลด้วยไทยล้วนหมายรักสามัคคี
Output positions: [(0, 5), (6, 8), (9, 11), (12, 21), (22, 25), (26, 30), (31, 33), (34, 37), (38, 42), (43, 45), (46, 48), (49, 51), (52, 54), (55, 57), (58, 61), (62, 65), (66, 69), (70, 74), (75, 77), (78, 84), (85, 88), (89, 91), (92, 95), (96, 99), (100, 102), (103, 109)]
Token amount: 26
ประเทศ|ไทย|รวม|เลือดเนื้อ|ชาติ|เชื้อ|ไทย|เป็น|ประชา|รัฐ|ไผท|ของ|ไทย|ทุก|ส่วน|อยู่|ดำรง|คงไว้|ได้|ทั้งมวล|ด้วย|ไทย|ล้วน|หมาย|รัก|สามัคคี


### Part 3.2) Use PyThaiNLP `word_tokenize` with custom dictionary

Try tokenizing the following text with `word_tokenize` in `newmm` algorithm and default real dictionary.

In [20]:
text = "นัดกินกันตอนไหนก็ได้ที่สามย่านมิตรทาวน์"

####FILL CODE HERE####

custom_dict = Trie(thai_vocab)
token = word_tokenize(text, custom_dict=custom_dict, engine="newmm")
print(token)

######################

['นัด', 'กินกัน', 'ตอน', 'ไหน', 'ก็', 'ได้ที่', 'สามย่าน', 'มิตร', 'ทาวน์']


Add 'สามย่านมิตรทาวน์' into dictionary and then tokenize again

In [23]:
####FILL CODE HERE####

new_thai_vocab = thai_vocab + ["สามย่านมิตรทาวน์"]
new_custom_dict = Trie(new_thai_vocab)

token = word_tokenize(text, custom_dict=new_custom_dict, engine="newmm")
print(token)

######################

['นัด', 'กินกัน', 'ตอน', 'ไหน', 'ก็', 'ได้ที่', 'สามย่านมิตรทาวน์']


### <font color=blue>Question 3</font>
Using the code from part three only, how many “words” did you get when tokenizing this input text **after adding the new vocabs**.

Answer this question in question #3 in MyCourseVille. Also print out the answer in this notebook as well.

In [24]:
new_vocab = ["ดิสนีย์ออนไอซ์", "ตีกอล์ฟ", "ธรรมมะ"]
input_text = "อ๋อก็ว่าจะไปเรียนแต่งหน้านั่งสมาธิดำน้ำปลูกปะการังทำอาหารนวดสปาปลูกป่าดำนาดูดิสนีย์ออนไอซ์แรลลี่ตีกอล์ฟล่องเรือส่องสัตว์ช้อปปิ้งดูงิ้วดูละครเวทีดูคอนเสิร์ตดินเนอร์ทำขนมจัดดอกไม้เที่ยวตลาดน้ำเรียนถ่ายรูปดูกายกรรมชมเมืองเก่าเข้าสัมมนาทัวร์ธรรมมะเรียนเต้นแล้วก็ร้องเพลง"

thai_vocab = thai_vocab + new_vocab
out = maximal_matching(input_text)
print(f"Input text: {input_text}")
print(f"Output positions: {backtrack(out)}")
print(f"Token amount: {len(backtrack(out))}")

Input text: อ๋อก็ว่าจะไปเรียนแต่งหน้านั่งสมาธิดำน้ำปลูกปะการังทำอาหารนวดสปาปลูกป่าดำนาดูดิสนีย์ออนไอซ์แรลลี่ตีกอล์ฟล่องเรือส่องสัตว์ช้อปปิ้งดูงิ้วดูละครเวทีดูคอนเสิร์ตดินเนอร์ทำขนมจัดดอกไม้เที่ยวตลาดน้ำเรียนถ่ายรูปดูกายกรรมชมเมืองเก่าเข้าสัมมนาทัวร์ธรรมมะเรียนเต้นแล้วก็ร้องเพลง
Output positions: [(0, 2), (3, 4), (5, 7), (8, 9), (10, 11), (12, 16), (17, 24), (25, 33), (34, 38), (39, 42), (43, 49), (50, 56), (57, 59), (60, 62), (63, 69), (70, 73), (74, 75), (76, 89), (90, 95), (96, 102), (103, 110), (111, 119), (120, 127), (128, 129), (130, 133), (134, 135), (136, 143), (144, 145), (146, 154), (155, 162), (163, 164), (165, 167), (168, 170), (171, 176), (177, 182), (183, 189), (190, 194), (195, 201), (202, 203), (204, 210), (211, 212), (213, 217), (218, 221), (222, 225), (226, 231), (232, 236), (237, 242), (243, 247), (248, 251), (252, 257), (258, 265)]
Token amount: 51


## Part 4) Use maximal matching on real dataset

To complete this exercise, we will use the maximal matching algorithm on NECTEC's BEST corpus.

The corpus has a structure of characters with target whether it's a beginning of a word (True/False).

In [27]:
# Download dataset
%pip install -q gdown
!gdown "1EcrlXYUyIEM3aeIJse6nPpiv_UjSKgoU&confirm=t"

Downloading...
From: https://drive.google.com/uc?id=1EcrlXYUyIEM3aeIJse6nPpiv_UjSKgoU&confirm=t
To: /Users/jirayuwat/Desktop/2110572-NLP/homework1/corpora.tar.gz
100%|████████████████████████████████████████| 121M/121M [00:06<00:00, 18.1MB/s]


In [ ]:
!tar xvf corpora.tar.gz

In [31]:
%pip install -q pandas
import pandas as pd
import os

In [32]:
# Path to the preprocessed data
best_processed_path = "corpora/BEST"
option = "test"

df = []
# article types in BEST corpus
article_types = ["article", "encyclopedia", "news", "novel"]
for article_type in article_types:
    df.append(pd.read_csv(os.path.join(best_processed_path, option, "df_best_{}_{}.csv".format(article_type, option))))
df = pd.concat(df)
df

,char,target
0,ป,True
1,ฏ,False
2,ิ,False
3,ร,False
4,ู,False
...,...,...
644911,ห,False
644912,น,False
644913,ม,True
644914,า,False


In [33]:
len(df)

2271932

In [34]:
# Some text in this corpus
all_text = "".join(df["char"].tolist())
all_text[:1000]

'ปฏิรูปการศึกษา : มุมมองทางกระบวนทัศน์และบริบทสังคมไทยThe Reformation of Eucation from A Thai Perspectiveกระบวนทัศน์และวิธีคิดแบบแยกส่วน ลดส่วน ได้ทำให้"การศึกษาเรียนรู้"ใน หลายทศวรรษที่ผ่านมา กลายเป็นเรื่องของนักวิชาการด้านศึกษาศาสตร์ ครุศาสตร์ หรือเป็นเรื่องของโรงเรียน ครูอาจารย์ กระทรวงศึกษาธิการ ทบวงมหาวิทยาลัยฯ มาอย่างต่อเนื่องยาวนาน (เหมือนกับที่เรื่องสุขภาพเป็นเรื่องของแพทย์และโรงพยาบาล) การจัดการศึกษาภายใต้กระบวนทัศน์และวิธีคิดแบบดังกล่าวของรัฐ ได้ถูกวิพากษ์วิจารณ์และตกเป็นจำเลยจากวิกฤตการณ์ทางสังคมมากมาย อันสะท้อนถึงความล้มเหลวของการจัดการศึกษาเพื่อพัฒนามนุษย์ (ปัญหาศีลธรรมเสื่อมถอย ยาเสพติด การขาดจิตสำนึกทางสังคม ฯลฯ) ซึ่งสังคมร่วมกันสรุปว่า เกิดจากความล้มเหลวของระบบการศึกษาในกระบวนทัศน์แบบแยกส่วน นำมาสู่การปฏิรูปการศึกษาที่กำลังดำเนินการอยู่ในปัจจุบัน ด้วยเป้าหมายเพื่อสร้างการเรียนรู้แบบองค์รวม ที่จะทำให้"ผู้เรียนเก่ง-ดี-มีความสุข"คำถามที่ผู้เขียนสนใจในการปฏิรูปการศึกษาที่ดำเนินการในปัจจุบัน คือ๑.การปฏิรูปการศึกษาในปัจจุบันดำเนินการภายใต้กระบวนทัศน์แบบบูรณาการ (องค์รวม)ตามที

### <font color=blue>Question 4</font>
Using PyThaiNLP `newmm`, how many words did you get in the BEST corpus (test)? [Runtime is around 7 mins] What are the accuracy, f1, precision, recall scores for each character?

Answer this question in question #4 in MyCourseVille. Also print out the answer in this notebook as well.

_Question: What main metric should we look at? Why?_

In [37]:
####FILL CODE HERE####

# Tokenize the text
tokens = word_tokenize(all_text, engine="newmm")
print(f"First 10 tokens: {tokens[:10]}")
print(f"Total tokens: {len(tokens)}")

######################

First 10 tokens: ['ปฏิรูป', 'การศึกษา', ' ', ':', ' ', 'มุมมอง', 'ทาง', 'กระบวนทัศน์', 'และ', 'บริบท']
Total tokens: 569631


In [38]:
def convert_to_character(_tokens):
    char_list = [0] * len("".join(_tokens))
    char_count = 0
    for word in _tokens:
        char_list[char_count] = 1
        char_count += len(word)
    return char_list


chars = convert_to_character(tokens)
chars[:20]

[1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0]

In [40]:
%pip install -q scikit-learn
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

####FILL CODE HERE####

my_tokens_flag = convert_to_character(tokens)
true_tokens_flag = df["target"].astype(int).tolist()

f1 = f1_score(true_tokens_flag, my_tokens_flag)
precision = precision_score(true_tokens_flag, my_tokens_flag)
recall = recall_score(true_tokens_flag, my_tokens_flag)
accuracy = accuracy_score(true_tokens_flag, my_tokens_flag)

print(f"F1 score: {f1}")
print(f"Precision score: {precision}")
print(f"Recall score: {recall}")
print(f"Accuracy score: {accuracy}")

######################

1858.62s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Note: you may need to restart the kernel to use updated packages.
F1 score: 0.8925828735253718
Precision score: 0.9422661336900555
Recall score: 0.8478765332638281
Accuracy score: 0.9431373826329309
